# Verse Interpreter development

Essential installs:
- `pip3 install .....`


This version is used to test the first steps for the verse interpreter. The final version will be used as a full python file.

In [3]:
import string
import math
from enum import Enum

## Setting Token Enum for testing purpose

In [4]:
class TokenTypes(Enum):
    # Data
    Integer = int

    # Aritmetics
    Plus = "+"
    Minus = "-"
    Multiply = "*"
    Divide = "/"

    # Mehtods
    For = "for"
    LBracket = "("
    RBracket = ")"

    # Else
    EOF = None

In [115]:
list(TokenTypes)

[<TokenTypes.Integer: <class 'int'>>,
 <TokenTypes.Plus: '+'>,
 <TokenTypes.Minus: '-'>,
 <TokenTypes.Multiply: '*'>,
 <TokenTypes.Divide: '/'>,
 <TokenTypes.For: 'for'>,
 <TokenTypes.LBracket: '('>,
 <TokenTypes.RBracket: ')'>,
 <TokenTypes.EOF: None>]

In [5]:
class Token:
    def __init__(self, type: TokenTypes, value) -> None:
        self.type = type
        self.value = value

## Lexer

In [8]:
class lexicon:
    def __init__(self, input: string):
        self.input = input
        self.index = 0
        self.current_char = self.input[self.index]

    def error():
        raise Exception('Invalid character.')
    
    # moves the pointer a character forward
    def forward(self, index) -> tuple[str,int]:
        index += 1

        # check if index out of range
        if (index >= len(self.input)):
            return None, index
        
        return self.input[index],index
    
    def future_token(self, index) -> tuple[Token,int]:
        index += 1

        # check if index out of range
        if (index >= len(self.input)):
            return None,index
        
        return self.get_token(self.input[index], index)
    
    def get_int(self, index: int) -> tuple[int,int]:
        result = ''

        # checks if there are multiple digits
        while True:
            if index < len(self.input) and self.input[index] != None and self.input[index].isdigit():
                result += self.input[index]
                _,index = self.forward(index)
            else:
                break

        return int(result),index
    
    def get_token(self, char: string, index: int) -> tuple[Token,int]:
        if char is None:
            return Token(TokenTypes.EOF, None),index

        if char.isdigit():
            result,index = self.get_int(index)
            return Token(TokenTypes.Integer, result),index

        if char == TokenTypes.Divide.value:
            return Token(TokenTypes.Divide, TokenTypes.Divide.value),index
        
        if char == TokenTypes.Multiply.value:
            return Token(TokenTypes.Multiply, TokenTypes.Multiply.value),index
        
        if char == TokenTypes.Plus.value:
            return Token(TokenTypes.Plus, TokenTypes.Plus.value),index
        
        if char == TokenTypes.Minus.value:
            return Token(TokenTypes.Minus, TokenTypes.Minus.value),index
        
        if char == TokenTypes.LBracket.value:
            return Token(TokenTypes.LBracket, TokenTypes.LBracket.value),index
        
        if char == TokenTypes.RBracket.value:
            return Token(TokenTypes.RBracket, TokenTypes.LBracket.value),index
        
        if char == ' ':
            char,index = self.forward(index)
            return self.get_token(char, index)
        
        return Token(TokenTypes.EOF, None),index

## Testing Lexer Types

In [9]:
lexer = lexicon("723 + 2 - 5 * 1235 / 100")

while lexer.current_char is not None:
    token,lexer.index = lexer.get_token(lexer.current_char, lexer.index)
    print(str(token.value) + " is of the tokentype: " + str(token.type))
    future_token,index = lexer.future_token(lexer.index)
    if future_token is not None:
        print("Future:" + str(future_token.type) + "\n")
    lexer.current_char, lexer.index = lexer.forward(lexer.index)

723 is of the tokentype: TokenTypes.Integer
Future:TokenTypes.Plus

+ is of the tokentype: TokenTypes.Plus
Future:TokenTypes.Integer

2 is of the tokentype: TokenTypes.Integer
Future:TokenTypes.Minus

- is of the tokentype: TokenTypes.Minus
Future:TokenTypes.Integer

5 is of the tokentype: TokenTypes.Integer
Future:TokenTypes.Multiply

* is of the tokentype: TokenTypes.Multiply
Future:TokenTypes.Integer

1235 is of the tokentype: TokenTypes.Integer
Future:TokenTypes.Divide

/ is of the tokentype: TokenTypes.Divide
Future:TokenTypes.Integer

100 is of the tokentype: TokenTypes.Integer


## Syntax Tree Nodes

In [48]:
class BaseNode:
    def __init__(self, token) -> None:
        self.token = token

class NumberNode(BaseNode):
    def __init__(self, token) -> None:
        super().__init__(token)
        self.value = token.value

class OperatorNode(BaseNode):
    def __init__(self, token, leftNode: BaseNode, rightNode: BaseNode) -> None:
        super().__init__(token)
        self.leftNode = leftNode
        self.rightNode = rightNode

## Parser

In [22]:
class Parser:
    def __init__(self, lexer: lexicon) -> None:
        self.lexer = lexer
        self.current_token, self.index = lexer.get_token(self.lexer.current_char, self.lexer.index)
    
    def forward(self) -> None:
        self.lexer.current_char, self.lexer.index = self.lexer.forward(self.index)
        self.current_token, self.index = lexer.get_token(self.lexer.current_char, self.lexer.index)
    
    def expr(self):
        leftNode = self.term()
        token = self.current_token

        if token.type == TokenTypes.Plus or token.type == TokenTypes.Minus:
            self.forward()
        
            rightNode = self.term()
            return OperatorNode(token, leftNode, rightNode)
        
        return leftNode

    def term(self):
        leftNode = self.factor()
        token = self.current_token

        if token.type == TokenTypes.Multiply or token.type == TokenTypes.Divide:
            self.forward()
        
            rightNode = self.factor()
            return OperatorNode(token, leftNode,rightNode)
        
        return leftNode
    

    def factor(self):
        token = self.current_token

        if token.type == TokenTypes.Integer:
            self.forward()
            return NumberNode(token)
        
        if token.type == TokenTypes.LBracket:
            self.forward()
            node = self.expr()
            self.forward()
            return node
        
        return token
    
    def parse(self):
        return self.expr()

In [46]:
class Interpreter:
    def __init__(self, parser):
        self.parser = parser

    def Ovisitor(self, node: OperatorNode):
        if node.token.type == TokenTypes.Plus:
            return self.visit(node.leftNode) + self.visit(node.rightNode)
        elif node.token.type == TokenTypes.Minus:
            return self.visit(node.leftNode) - self.visit(node.rightNode)
        elif node.token.type == TokenTypes.Multiply:
            return self.visit(node.leftNode) * self.visit(node.rightNode)
        elif node.token.type == TokenTypes.Divide:
            return self.visit(node.leftNode) // self.visit(node.rightNode)

    def Nvisitor(self, node: NumberNode):
        return node.value

    def interpret(self):
        tree = self.parser.parse()
        return self.visit(tree)
    
    def visit(self, node: BaseNode):
        if type(node) == OperatorNode:
            return self.Ovisitor(node)
        
        return self.Nvisitor(node)

In [49]:
text = "3 * (2 + 4)"
lexer = lexicon(text)
parser = Parser(lexer)
interpreter = Interpreter(parser)
result = interpreter.interpret()
result

18